In [230]:
def is_valid_char(c):
    ## modify it to be faster
    # h = '{:06X}'.format(ord(c))
    # if ord('a') <= ord(c.lower()) and ord(c) <= ord('z'):  
    oc = ord(c)
    if 97 <= oc and oc <= 122:
        # latin character
        # print(c, 'is latin lower')
        return True
    if 65 <= oc and oc <= 90:
        # latin character
        # print(c, 'is latin upper')
        return True
    # if h.startswith('000E'):
    if (oc & 0xFFFF00) == 0x000E00:
        # is thai character
        return True
    return False

def valid_words(word):
    result = []
    buffer = None
    for c in word:
        if is_valid_char(c):
            if buffer is None:
                buffer = c
                result.append(buffer)
            else:
                buffer += c
                result[-1] = buffer
        else:
            if buffer is not None:
                buffer = None
    return result

ss = ['sinthorn summary ', '11', '/', '10', '/', '2560', ' ', ':', ' ', 'ต่าง', 'ชาติ', 'ซื้อ', 'สุทธิ', ' ', '1', ',', '600', '.', '57', ' ', 'ลบ.']
# valid_words('อากาศ 😭_😭_😭 ร้อน_กับ ADVANC ad')
valid_words(' '.join(ss))

['sinthorn', 'summary', 'ต่าง', 'ชาติ', 'ซื้อ', 'สุทธิ', 'ลบ']

In [246]:
def file_sentences(path):
    import ujson
    
    
    with open(path) as f:
        try:
            topic = ujson.loads(f.read())
            attributes = [topic['message'], topic['title']] + [comment['text'] for comment in topic['comments']]
            for sentences in attributes:
                for sentence in sentences:
                    sentence = [word.lower() for word in sentence]
                    sentence2 = []
                    for word in sentence:
                        words = valid_words(word)
                        for valid_word in words:
                            sentence2.append(valid_word)

                    if len(sentence2) > 0:
                        yield(sentence2)
        except ValueError as e:
            pass

def corpus_sentences(size_limit = 10000, seed=None):
    import glob
    import itertools
    import random
    if seed is not None:
        random.seed(seed)
    paths = glob.glob('tokenized/*/*/*/*.json')
    random.shuffle(paths)
    sample_paths = paths
    total_size = 0
    for path in sample_paths:
        for sentence in file_sentences(path):
            total_size += sum([len(word) for word in sentence])
            yield(sentence)
        if total_size > size_limit:
            break
            

for sentence in corpus_sentences(seed=10):
    print(sentence)

['ตอน', 'ลง', 'จะ', 'ลง', 'แบบ', 'ลิฟต์ไม๊เนี่ย']
['paf', 'มัน', 'เล่น', 'อะไร', 'กัน']
['ไม่', 'รู้', 'เหมือน', 'กัน', 'ครับ']
['หุ้น', 'ของ', 'คน', 'หวัง', 'รวย', 'ก่อน', 'ปี', 'ใหม่', 'ละมั่ง', 'ครับ']
['paf', 'บวก', 'สวน', 'ตลาดฯ', 'กว่า', 'โบรก', 'ฯ', 'แนะ', 'ขาย', 'ที่', 'แนวต้าน', 'บ']
['สำนักข่าว', 'อีไฟแนนซ์', 'ไทย', 'ธ', 'ค', 'น']
['ผู้', 'สื่อ', 'ข่าว', 'รายงาน', 'ความ', 'เคลื่อนไหว', 'ราคา', 'หุ้น', 'บริษัท', 'แพนเอเซียฟุตแวร์', 'จำกัด', 'มหาชน', 'หรือ', 'paf', 'ปรับ', 'ตัว', 'เพิ่ม', 'ขึ้น', 'กว่า', 'และ', 'มี', 'มูลค่า', 'การ', 'ซื้อขาย', 'หนา', 'แน่น', 'เป็น', 'อันดับ', 'วัน', 'นี้', 'โดย', 'ล่าสุด', 'เวลา', 'ประมาณ', 'น', 'ราคา', 'หุ้น', 'อยู่', 'ที่', 'บาท', 'เพิ่ม', 'ขึ้น', 'บาท', 'หรือ', 'มูลค่า', 'การ', 'ซื้อขาย', 'ล้าน', 'บาท']
['ก่อนหน้า', 'นี้', 'paf', 'ชี้แจง', 'ตลาด', 'หลักทรัพย์แห่งประเทศไทย', 'ตลท', 'ว่า', 'ไม่', 'มี', 'พัฒนาการ', 'ใด', 'ที่', 'เป็น', 'นัย', 'สำคัญ', 'ที่', 'จะ', 'เปิดเผย']
['ขณะ', 'ที่', 'บท', 'วิเคราะห์', 'บล', 'ดีบีเอสวิคเคอร์ส', 'แนะ', 'ข

In [6]:
import random
import sys
seed = random.randrange(sys.maxsize)
size_limit = 50000000

In [7]:
%%time
with open('corpus.cache.txt','w') as f:
    for sentence in corpus_sentences(seed=seed, size_limit=size_limit):
        f.write(' '.join(sentence))
        f.write("\n")

CPU times: user 1min 17s, sys: 1.62 s, total: 1min 18s
Wall time: 2min 7s


In [8]:
%%time

def cached_sentences():
    with open('corpus.cache.txt') as f:
        for line in f:
            sentence = line.split()
            if len(sentence) > 0:
                yield(sentence)
                
for s in cached_sentences():
    pass

CPU times: user 1.76 s, sys: 96 ms, total: 1.85 s
Wall time: 13.8 s


In [9]:
%%time
import gensim
# biphrases = gensim.models.phrases.Phrases(corpus_sentences(size_limit=size_limit, seed=seed))
biphrases = gensim.models.phrases.Phrases(cached_sentences())
bigram_transformer = gensim.models.phrases.Phraser(biphrases)
biphrases = None

triphrases = gensim.models.phrases.Phrases(bigram_transformer[cached_sentences()])
trigram_transformer = gensim.models.phrases.Phraser(triphrases)
triphrases = None

CPU times: user 2min 28s, sys: 496 ms, total: 2min 29s
Wall time: 2min 30s


In [10]:
bigram_transformer.save('bigram.transformer.bin')
trigram_transformer.save('trigram.transformer.bin')

In [ ]:
# calculation time log
# trigram Wall time: 4min 12s
# word vector Wall time: 3min 38s

In [171]:
def trisentences():
    return trigram_transformer[bigram_transformer[cached_sentences()]]

In [172]:
%%time
with open('corpus.3gram.cache.txt','w') as f:
    for sentence in trisentences():
        f.write(' '.join(sentence))
        f.write("\n")

CPU times: user 1min 23s, sys: 448 ms, total: 1min 23s
Wall time: 1min 25s


In [258]:
def tri_cached_sentences():
    with open('corpus.3gram.cache.txt') as f:
        for line in f:
            sentence = line.split()
            if len(sentence) > 0:
                yield(sentence)
                
for s in tri_cached_sentences():
    pass

In [259]:
# make it iterator, not just generator
class TriCachedCorpus(object):
    def __iter__(self):
        for sentence in tri_cached_sentences():
            yield(sentence)

In [328]:
bigram_transformer = None
trigram_transformer = None

In [260]:
import itertools
for s in itertools.islice(TriCachedCorpus(), 100):
    print(s)

['eva_air', 'หัก', 'จาก', 'บัตร_เดบิต', 'ได้', 'ไหม', 'ค่ะ']
['เหมือน', 'ใช้', 'บัตร', 'ของ', 'กสิกร', 'มี', 'ส่วน', 'ลด', 'ด้วย', 'ครับ']
['สวัสดี', 'ครับ', 'ผอง_เพื่อน_เหล่า', 'นัก_รบ_กอง_ทุน', 'และ', 'พราน_หุ้น_ทุก_ท่าน', 'เมื่อ_วาน', 'ปู่เซ็ต', 'ลงแดง', 'ดัชนี', 'ปิด', 'ที่', 'จุด', 'หรือ', 'เหตุ', 'จาก', 'ความ', 'ไม่', 'แน่นอน', 'ด้าน', 'นโยบาย', 'งบ_ประมาณ', 'ฝั่ง_สหรัฐ', 'โน่น', 'กระทบ', 'ไป', 'ทั่ว_โลก', 'ทำ', 'ให้', 'โปรเจค_ddt', 'ได้', 'ส่วน', 'ลด', 'ไป', 'พอสมควร', 'ดีใจ', 'มั๊ย', 'ครับ', 'ด่าน', 'สมรภูมิ', 'อื่น', 'ๆ', 'ก็', 'พลอย', 'ติด_ลบ', 'ไป', 'ด้วย', 'ตาม', 'บรรยากาศ', 'การ', 'ลง_ทุน', 'ที่', 'เต็ม', 'ไป', 'ด้วย', 'ความ_กังวล', 'ใน', 'ภาวะ', 'เช่น', 'นี้', 'นัก_รบ', 'ควร', 'อยู่', 'เฉย_ๆ', 'คือ', 'เป็น', 'สถานะ', 'รอ', 'นั่น_เอง', 'เมื่อ_วัน_เสาร์', 'มี', 'เวลา_ว่าง', 'เล็ก', 'ๆ', 'ไล่', 'ตรวจ', 'ดู', 'ราย_ชื่อ', 'หุ้น', 'ใน', 'กลุ่มดัชนี', 'set', 'พยายาม', 'มอง', 'หา', 'หุ้น', 'ดี', 'ราคา', 'ไม่', 'ถูก', 'เผื่อ', 'จะ', 'ได้', 'มุมมอง', 'อะไร', 'เพิ่มเติม', 'บ้าง', 'พ

In [270]:
%%time
import datetime
print(datetime.datetime.now())
wvmodel = gensim.models.Word2Vec(TriCachedCorpus(), size=100, window=10, sg=1, iter=10)

wvmodel.train(TriCachedCorpus(), total_examples=wvmodel.corpus_count, epochs=wvmodel.epochs)

2018-08-15 16:23:50.819916
CPU times: user 30min 19s, sys: 3.57 s, total: 30min 22s
Wall time: 30min 42s


In [262]:
pairs = [
    ('คัท', 'cut'),
    ('ดอย', 'ติด_ดอย'),
    ('ดอย', 'ถัว'),
    ('ขาดทุน', 'ดอย'),
    ('ขาดทุน', 'กำไร'),
    ('floor', 'ดอย'),
    ('floor', 'ขาดทุน'),
    ('floor', 'คัท'),
    ('set', 'ปู่'),
]

for pair in pairs:
    sim = wvmodel.wv.similarity(pair[0], pair[1])
    print('{:5.2f}% {} {}'.format(sim * 100, pair[0], pair[1]))

76.60% คัท cut
81.96% ดอย ติด_ดอย
73.12% ดอย ถัว
40.12% ขาดทุน ดอย
53.43% ขาดทุน กำไร
46.96% floor ดอย
37.01% floor ขาดทุน
47.82% floor คัท
55.03% set ปู่


/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [359]:
wvmodel.wv.most_similar('เขียว')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('แดง', 0.9008665680885315),
 ('พี่หมาก', 0.7620635628700256),
 ('เด้ง_ดึ๋ง', 0.7315344214439392),
 ('พี่we_love_thailand', 0.7221149206161499),
 ('แดง_แจ๋', 0.7144074440002441),
 ('ลงแดง', 0.7134711146354675),
 ('เอเชียแดง', 0.7097564339637756),
 ('เขียว_อ่อน', 0.707740306854248),
 ('เขียว_เข้ม', 0.7052264213562012),
 ('ออกแดง', 0.6860056519508362)]

In [264]:
wvmodel.wv.most_similar('ติดดอย')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ติด_ดอย', 0.8950231075286865),
 ('ดอย', 0.8065218925476074),
 ('ขาย_หมู', 0.782018780708313),
 ('คัท', 0.7539661526679993),
 ('ถัว', 0.749467134475708),
 ('ทน_ถือ', 0.7403892278671265),
 ('vi_จำเป็น', 0.7181094884872437),
 ('หมู_หก', 0.7087310552597046),
 ('เม่า', 0.7008231282234192),
 ('ดอยหนัก', 0.6987157464027405)]

In [265]:
wvmodel.wv.most_similar('ปู่')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dow_เขียว', 0.6977851390838623),
 ('estprev_close', 0.691551685333252),
 ('ดาวโจร', 0.6877351999282837),
 ('เขียว_แสบ_ตา', 0.671231746673584),
 ('ปู๋', 0.6710683107376099),
 ('p_m_previous', 0.6679481267929077),
 ('ปู่_set', 0.6670135259628296),
 ('close_day_low_day', 0.6635088920593262),
 ('แดง_แจ๋', 0.6617985367774963),
 ('เนี่ยยยยย', 0.65919029712677)]

In [268]:
wvmodel.wv.most_similar('กำไร')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ขาด_ทุน', 0.7407156825065613),
 ('q_q_y_y', 0.6918238401412964),
 ('สุทธินิวไฮ', 0.6777358651161194),
 ('หลังค่า', 0.6692806482315063),
 ('สต๊อกน้ำมัน', 0.6682887077331543),
 ('ผล_ประกอบ_การ', 0.6618397235870361),
 ('นิวไฮต่อเนื่อง', 0.6616955995559692),
 ('คุณขาด_ทุน', 0.655654788017273),
 ('กำไร_สุทธิ', 0.649263322353363),
 ('จ่อ_บุ๊ก_กำไร', 0.6485243439674377)]

In [266]:
wvmodel.wv.most_similar('ถัว')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('คัท', 0.7897443771362305),
 ('ถัว_จน', 0.7791475653648376),
 ('ซื้อถัว', 0.7631266117095947),
 ('ถือกิน_ปัน_ผล', 0.7622551918029785),
 ('ติดดอย', 0.7494672536849976),
 ('ไม้_แรก', 0.7355560064315796),
 ('ติด_ดอย', 0.7347210645675659),
 ('ดอย', 0.7312475442886353),
 ('คัต', 0.7222053408622742),
 ('ทน_ถือ', 0.7122272849082947)]

In [410]:
wv.most_similar('ทน_รวย')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ทน_ถือ', 0.6762784719467163),
 ('ติดดอย', 0.6643658876419067),
 ('ปิด_แกป', 0.6591575145721436),
 ('ติด_ดอย', 0.6524772047996521),
 ('หนายยย', 0.6524191498756409),
 ('ขา_put', 0.6433411836624146),
 ('ขา_ช้อต', 0.640041708946228),
 ('ดอย', 0.6352077722549438),
 ('เพ่กอง', 0.6343613862991333),
 ('ฮ๊าฟ', 0.6329175233840942)]

In [409]:
wvmodel.wv.most_similar('กระดิก_ตีน')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('เยี่ยวเหนียว', 0.673949658870697),
 ('มีเซียน', 0.6632639169692993),
 ('ซักตัว', 0.6515215635299683),
 ('หลับ_ตา_จิ้ม', 0.6429361701011658),
 ('แถไป', 0.6382111310958862),
 ('นนนนนนนนนนน', 0.6377637982368469),
 ('คุณพระช่วย', 0.6287404298782349),
 ('เทหมด', 0.627719521522522),
 ('กองเค้า', 0.6269105672836304),
 ('แม่นจริง', 0.6259508728981018)]

In [406]:
wvmodel.wv.most_similar('ปิดจอ')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('เกาไข่', 0.6946024298667908),
 ('ดี_ก่า', 0.6813105344772339),
 ('งีบ', 0.6799486875534058),
 ('เหนี่อย', 0.6754287481307983),
 ('ตกเหว', 0.6748765707015991),
 ('ปิด_จอ', 0.6742295026779175),
 ('ทิตย์', 0.6655632257461548),
 ('เนี่ยยยย', 0.662006139755249),
 ('นอน', 0.6580524444580078),
 ('เล่น_เดย์', 0.6518649458885193)]

In [267]:
wvmodel.wv.most_similar('ล้าง_พอร์ท')

/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ล้าง_port', 0.6879936456680298),
 ('ล้าง_ปอด', 0.683260440826416),
 ('คัท', 0.6706336736679077),
 ('ถัว_จน', 0.6682713627815247),
 ('ทันครับ', 0.6624603271484375),
 ('รินขาย', 0.6522741317749023),
 ('กระโดด_หนี', 0.6517815589904785),
 ('เล่น_ปั่น_แปะ', 0.6509268879890442),
 ('คัทลอส', 0.650353729724884),
 ('สวนเทรน', 0.6500637531280518)]

In [119]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="9"

In [271]:
wvmodel.wv.save('word-vector.window10.bin')

# PCA Time

In [343]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('word-vector.window10.bin')

In [344]:
bad_sentiment_words = ['ดอย', 'ขาดทุน', 'แดง', 'ล้าง_ปอด', 'คัท', 'ติดดอย', 'new_low', 'จน']
good_sentiment_words = ['กำไร', 'เขียว', 'ลิ่ง', 'รวย', 'กิน_เหลา', 'new_high']

In [345]:
bad_sentiment_words = ['ดอย', 'ล้าง_ปอด', 'คัท', 'ติดดอย', 'เครียด']
good_sentiment_words = ['ลิ่ง', 'รวย', 'กิน_เหลา']

In [346]:
X = [
    wv[word]
    for word
    in bad_sentiment_words + good_sentiment_words
]

In [347]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=1)
pca.fit(X)
X1 = pca.transform(X)

In [348]:
allwords = bad_sentiment_words + good_sentiment_words
data = [
    {
        'word' : allwords[i],
        'score' : X1[i][0],
    }
    for i
    in range(len(X1))
]
data.sort(key=lambda item: item['score'])

In [349]:
for item in data:
    print('{:5.2f} {}'.format(item['score'], item['word']))

-1.02 ติดดอย
-0.81 คัท
-0.73 ดอย
-0.67 เครียด
-0.27 ลิ่ง
-0.11 ล้าง_ปอด
 0.50 รวย
 3.11 กิน_เหลา


In [358]:
bad_sentiment_words = ['ดอย', 'ขาดทุน', 'แดง', 'ล้าง_ปอด', 'ล้าง_พอร์ท', 'คัท', 'ติดดอย', 'new_low', 'จน', 'ทุบ']
good_sentiment_words = ['กำไร', 'เขียว', 'ลิ่ง', 'รวย', 'กิน_เหลา', 'new_high', 'เด้ง', 'สูงสุด', 'โลภ', 'เปิด_port']
allwords = bad_sentiment_words + good_sentiment_words
X = [
    wv[word]
    for word
    in bad_sentiment_words + good_sentiment_words
]
X1 = pca.transform(X)
data = [
    {
        'word' : allwords[i],
        'score' : X1[i][0],
    }
    for i
    in range(len(X1))
]
data.sort(key=lambda item: item['score'])
for item in data:
    print('{:5.2f} {}'.format(item['score'], item['word']))

-1.02 ติดดอย
-0.81 คัท
-0.73 ดอย
-0.52 โลภ
-0.38 ขาดทุน
-0.28 new_low
-0.27 ลิ่ง
-0.19 สูงสุด
-0.12 แดง
-0.11 ล้าง_ปอด
-0.05 เปิด_port
-0.04 ทุบ
-0.04 จน
 0.03 กำไร
 0.05 เขียว
 0.12 เด้ง
 0.21 ล้าง_พอร์ท
 0.50 รวย
 0.56 new_high
 3.11 กิน_เหลา


# Good Day Bad Day Sample

In [102]:
## BADDD

import glob
import ujson

words = []
days = ['2016/1/7','2013/8/28', '2013/9/4']
for day in days:
    pattern = 'tokenized/{}/*json'.format(day)
    for path in glob.glob(pattern):
        sentences = file_sentences(path)
        sentences = trigram_transformer[bigram_transformer[sentences]]
        for sentence in sentences:
            for word in sentence:
                words.append(word)

In [103]:
bad_words = words

In [104]:
import collections
bow = collections.Counter(words)
for item in bow.most_common(1000):
    word = item[0]
    if word in wv:
        sent = pca.transform([wv[word]])[0][0]
        if sent < -1 or 1 < sent:
            print('{:7.4f}% {:5.2f} {}'.format(item[1]/len(words) * 100, sent, item[0]))

 0.6738% -1.06 เงิน
 0.1178% -1.29 หา
 0.0620% -1.19 ขาด_ทุน
 0.0362% -1.01 อัตรา
 0.0358% -1.07 พื้นฐาน
 0.0347% -1.16 ดอย
 0.0271% -1.04 vi
 0.0245% -1.05 e_b_e_b
 0.0228%  1.97 เขียว
 0.0217% -1.02 บริหาร
 0.0192%  1.51 แดง
 0.0181% -1.02 อาชีพ
 0.0181% -1.16 รู้จัก
 0.0162% -1.14 ออม
 0.0153% -1.03 ต้นทุน
 0.0149% -1.02 พัน_ล้าน_บาท
 0.0149% -1.05 ผล_ประโยชน์
 0.0147% -1.15 ธปท
 0.0147% -1.02 ดูแล
 0.0136% -1.19 โทษ
 0.0136% -1.06 ชาว_นา
 0.0132% -1.20 ประสบการณ์
 0.0130% -1.07 จ้าง
 0.0128% -2.53 link_http_goo_gl
 0.0119% -1.19 ฐานะ
 0.0117% -1.03 คอย
 0.0109% -1.00 หนี้สิน
 0.0102% -1.38 ราคา_เป้าหมาย
 0.0096% -1.33 กยศ


In [105]:
## GOOD

import glob
import ujson

words = []
days = ['2013/5/20','2013/5/28', '2013/3/15', '2013/5/21']
for day in days:
    pattern = 'tokenized/{}/*json'.format(day)
    for path in glob.glob(pattern):
        sentences = file_sentences(path)
        sentences = trigram_transformer[bigram_transformer[sentences]]
        for sentence in sentences:
            for word in sentence:
                words.append(word)

In [106]:
good_words = words

In [107]:
import collections
bow = collections.Counter(words)
for item in bow.most_common():
    word = item[0]
    if word in wv:
        sent = pca.transform([wv[word]])[0][0]
        if sent < -1 or 1 < sent:
            print('{:7.4f}% {:5.2f} {}'.format(item[1]/len(words) * 100, sent, item[0]))

 0.7248% -1.06 เงิน
 0.1199% -1.29 หา
 0.0579% -1.19 ขาด_ทุน
 0.0435% -1.07 พื้นฐาน
 0.0418% -1.01 อัตรา
 0.0338% -1.16 ดอย
 0.0314% -1.02 บริหาร
 0.0294% -1.02 ดูแล
 0.0275% -1.00 บริษัทฯ
 0.0210%  1.97 เขียว
 0.0195% -1.15 ธปท
 0.0188% -1.16 รู้จัก
 0.0186% -1.02 พัน_ล้าน_บาท
 0.0176%  1.51 แดง
 0.0174% -1.20 ประสบการณ์
 0.0166% -1.14 ออม
 0.0162% -1.10 อาจารย์
 0.0138% -1.02 ครอบครัว
 0.0135% -1.02 ตำแหน่ง
 0.0135% -1.03 ต้นทุน
 0.0116% -1.26 ลา_ออก
 0.0116%  1.20 จุด_เพิ่ม_ขึ้น
 0.0111% -1.05 e_b_e_b
 0.0109%  1.13 ได้_ชีวิต_คุณสบาย
 0.0109% -1.19 ฐานะ
 0.0108% -1.02 อาชีพ
 0.0104% -2.53 link_http_goo_gl
 0.0104% -1.30 หุ้น_forex
 0.0104% -1.11 เสีย_ภาษี
 0.0102% -1.00 ครู
 0.0096% -1.04 vi
 0.0092% -1.14 จัดสรร
 0.0090% -1.03 คอย
 0.0087%  1.20 เพิ่ม_ขึ้น_จุด
 0.0084% -1.11 bch
 0.0080% -1.00 หนี้สิน
 0.0079%  1.31 ปิด_บวก
 0.0079% -1.38 ราคา_เป้าหมาย
 0.0079% -1.06 รับผิดชอบ
 0.0079% -1.04 เรียนรู้
 0.0077% -1.05 ผล_ประโยชน์
 0.0068% -1.08 สอน_forex_exness
 0.0068% -1.22 ค้น_ใน
 

 0.0003% -1.06 หลวม_ตัว
 0.0003% -1.02 ค่า_โทรศัพท์
 0.0003% -1.17 วัย_เด็ก
 0.0003% -1.33 นพ
 0.0003% -1.28 โกยเบี้ย
 0.0003% -1.18 กระทู้_เก่า
 0.0003% -1.14 บวช
 0.0003% -1.16 กรรมการ_ผู้_บริหาร
 0.0003% -1.07 จ่อ_บุ๊ค_ราย
 0.0003% -1.19 ยกเลิก_สัญญา
 0.0003% -1.28 มือ_เก๋า
 0.0003% -1.25 เก้า
 0.0003%  1.53 แท่ง_เขียว
 0.0003% -1.18 ควบคุม_คุณภาพ
 0.0003% -1.03 capital_gain
 0.0003%  1.16 ลงแดง
 0.0003% -1.05 การงาน
 0.0003% -1.22 การบินไทย
 0.0003% -1.03 วาง_พิกัด
 0.0003% -1.06 ใจกล้า
 0.0003% -1.07 อดทน_รอ
 0.0003% -1.07 หน้าตัก
 0.0003% -1.21 money_management
 0.0003% -1.42 ใฝ่
 0.0003% -1.08 เที่ยว_ต่าง_ประเทศ
 0.0003% -1.04 สต๊อก
 0.0003% -1.21 อัด_ฉีด_งบ
 0.0003% -1.12 กลิ่นประทุม_บลจ_บัวหลวง
 0.0003% -1.16 บ่ม
 0.0003% -1.32 บริษัทหลักทรัพย์_บล
 0.0003% -1.18 อะไหล่
 0.0003% -1.08 วิธี_ทาง_เทคนิคอล
 0.0003%  1.32 bearish
 0.0003% -1.10 ธุรกิจ_ค้า_ปลีก
 0.0003% -1.00 กลัว_โดน_โกง
 0.0003% -1.02 แก้ไข_พ_ร
 0.0003% -1.17 ตาม_อัตภาพ
 0.0003% -1.09 วัคซีน
 0.0003% -1.01 กระทรวงศ

In [122]:
goodcount = {}
for w in good_words:
    goodcount[w] = goodcount.get(w, 0) + 1
    
badcount = {}
for w in bad_words:
    badcount[w] = badcount.get(w, 0) + 1

    
items = []
allwords = list(set(good_words + bad_words))
for k in allwords:
    item = {
        'word' : k,
        'count' : [
            goodcount.get(k, 0)/len(good_words),
            badcount.get(k, 0)/len(bad_words),
        ]
    }
    item['score'] = max(item['count'][0], item['count'][1]) / max(0.0001, min(item['count']))
    item['sent'] = 1 if item['count'][0] > item['count'][1] else -1
    items.append(item)
    
items.sort(key=lambda item: -item['score'])
items

[{'count': [2.2196951846274155e-05, 0.0007284965705278405],
  'score': 7.284965705278404,
  'sent': -1,
  'word': 'ยาง'},
 {'count': [0.0006778607602285261, 4.260213862735909e-05],
  'score': 6.778607602285261,
  'sent': 1,
  'word': 'พ_ค'},
 {'count': [1.7074578343287814e-05, 0.000513355770459677],
  'score': 5.13355770459677,
  'sent': -1,
  'word': 'ซีเรีย'},
 {'count': [3.927153018956197e-05, 0.0004260213862735909],
  'score': 4.260213862735909,
  'sent': -1,
  'word': 'ปลูก'},
 {'count': [0.00010756984356271323, 0.0004473224555872705],
  'score': 4.158437353555148,
  'sent': -1,
  'word': 'ข้าว'},
 {'count': [6.829831337315125e-06, 0.00038767946150896775],
  'score': 3.8767946150896773,
  'sent': -1,
  'word': 'ก_ย'},
 {'count': [0.0006522488927135945, 0.0001746687683721723],
  'score': 3.7342044533332204,
  'sent': 1,
  'word': 'วิ่ง'},
 {'count': [0.0003722258078836743, 4.260213862735909e-05],
  'score': 3.722258078836743,
  'sent': 1,
  'word': 'ค'},
 {'count': [9.5617638722411

In [156]:
import csv
with open('word-polarity.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['word','polar','size','positive','negative'])
    writer.writeheader()
    for item in items:
        row = {
            'word' : item['word'],
            'polar' : item['sent'],
            'size' : item['score'],
            'positive' : item['count'][0],
            'negative' : item['count'][1],
        }
        writer.writerow(row)

In [168]:
goodwords = ['ทะยาน', 'วิ่ง','แตะ','แข็ง','เขียว','ขา_ขึ้น','สูง_สุด', 'กำไร', 'แข็งแกร่ง', 'ปั่น', 'ขาย_หมู', 'call', 'long', 'ขาด_ทุน', 'เปิด_l', 'l', 'ลาก']
badwords = ['ดอย', 'หลุด','ร่วง','ตาย','แดง',  'ขา_ลง', 'ต่ำ_สุด', 'คัท', 'หนัก', 'ลบ', 'ตก', 'ร่วง_ลง', 'เลิก', 'เครียด', 'รอด', 'ทุบ', 'กังวล', 'put', 'short', 's', 'อ่อน', 'ไส้_แตก', 'เลือดสาด', 'ตบ']

In [169]:
sentwords = []
for w in goodwords:
    item = {'word' : w, 'sent' : 1}
    sentwords.append(item)
for w in badwords:
    item = {'word' : w, 'sent' : -1}
    sentwords.append(item)

In [236]:
sent_cache = {}

In [244]:
sent_cache

{'สมาคมค้า_ทองคำ': {'similarity': 0.36978504,
  'word': {'sent': -1, 'word': 'ร่วง_ลง'}},
 'แป๊ป': {'similarity': 0.52990305, 'word': {'sent': -1, 'word': 'ตบ'}},
 'ออนไลด์': {'similarity': 0.32509127,
  'word': {'sent': 1, 'word': 'แข็งแกร่ง'}},
 'ต้อง_เผชิญ_กับ': {'similarity': 0.4500733,
  'word': {'sent': -1, 'word': 'กังวล'}},
 'นิคม_อุตสาหกรรม': {'similarity': 0.2927236,
  'word': {'sent': 1, 'word': 'แข็งแกร่ง'}},
 'ลอส': {'similarity': 0.5785567, 'word': {'sent': -1, 'word': 'เลือดสาด'}},
 'ประสงค์': {'similarity': 0.27067077, 'word': {'sent': -1, 'word': 'กังวล'}},
 'ความ_รู้สึก': {'similarity': 0.54989415,
  'word': {'sent': -1, 'word': 'เครียด'}},
 'บาท_gf_m': {'similarity': 0.47313878,
  'word': {'sent': -1, 'word': 'ร่วง_ลง'}},
 'ู': {'similarity': 0.4198833, 'word': {'sent': -1, 'word': 'หลุด'}},
 'อะคับ': {'similarity': 0.3650016, 'word': {'sent': -1, 'word': 'เลือดสาด'}},
 'รัย': {'similarity': 0.4816944, 'word': {'sent': -1, 'word': 'เครียด'}},
 'กก': {'similarity': 0.

In [243]:
%%time


def best_sent_cache(word):
    if word in sent_cache:
        return sent_cache[word]
    
    items = [
        {
            'word': sentword,
            'similarity': wv.similarity(word, sentword['word'])
        }
        for sentword
        in sentwords
    ]
    bestsent = max(items, key=lambda item:item['similarity'])
    
    sent_cache[word] = bestsent
    return bestsent

def sent_for_day(day):
    pattern = 'tokenized/{}/*.json'.format(day)
    positive = 0
    negative = 0
    sent = 0
    
    for path in glob.glob(pattern):
        sentences = file_sentences(path)
        sentences = trigram_transformer[bigram_transformer[sentences]]
        
        for sentence in sentences:
            for word in sentence:
                if word in wv:
                    bestsent = best_sent_cache(word)
                    if bestsent['similarity'] > 0.7:
                        # print(word, bestsent)
                        sent += bestsent['word']['sent']
               
    
    return sent

score = sent_for_day('2013/5/20')
print(score)

341
CPU times: user 2.04 s, sys: 12 ms, total: 2.06 s
Wall time: 2.07 s


# Time to calculate

In [247]:
import glob
import random
import datetime
dirs = glob.glob('tokenized/*/*/*')
random.shuffle(dirs)
for i in range(len(dirs)):
    basedir = dirs[i]

    import os
    import re

    print(datetime.datetime.now(), i, len(dirs), basedir)
    sent_dir = basedir.replace('sentiment', 'trigram')
    sent_path = '/'.join([sent_dir, 'trigram.json'])
    if not os.path.exists(sent_path):
        day = re.findall(r'\d{4}\/\d{1,2}\/\d{1,2}', sent_dir)[0]
        sent = sent_for_day(day)
        if not os.path.exists(sent_dir):
            os.makedirs(sent_dir)
        with open(sent_path, 'w') as f:
            f.write('{}'.format(sent))
        print('sentimental score', sent)

2018-08-13 15:51:20.325756 0 2033 tokenized/2015/5/7
2018-08-13 15:51:20.326409 1 2033 tokenized/2015/10/17


/home/nc-user/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


sentimental score -55
2018-08-13 15:51:21.290517 2 2033 tokenized/2015/7/28
2018-08-13 15:51:21.290905 3 2033 tokenized/2014/5/13
sentimental score -61
2018-08-13 15:51:22.674507 4 2033 tokenized/2017/3/3
sentimental score 83
2018-08-13 15:51:23.734512 5 2033 tokenized/2018/6/19
sentimental score -259
2018-08-13 15:51:25.156644 6 2033 tokenized/2013/4/13
sentimental score -167
2018-08-13 15:51:26.212322 7 2033 tokenized/2014/5/17
sentimental score -41
2018-08-13 15:51:27.658259 8 2033 tokenized/2013/4/1
2018-08-13 15:51:27.658734 9 2033 tokenized/2015/11/22
2018-08-13 15:51:27.659119 10 2033 tokenized/2014/3/12
2018-08-13 15:51:27.659522 11 2033 tokenized/2017/6/27
sentimental score 104
2018-08-13 15:51:29.606714 12 2033 tokenized/2015/3/13
sentimental score 90
2018-08-13 15:51:31.336714 13 2033 tokenized/2018/4/28
sentimental score -19
2018-08-13 15:51:31.969503 14 2033 tokenized/2017/5/2
2018-08-13 15:51:31.970036 15 2033 tokenized/2017/2/10
sentimental score -58
2018-08-13 15:51:32.

sentimental score 64
2018-08-13 15:52:56.911682 124 2033 tokenized/2014/1/7
sentimental score 135
2018-08-13 15:52:59.380210 125 2033 tokenized/2013/5/4
sentimental score 100
2018-08-13 15:53:00.874458 126 2033 tokenized/2014/10/22
2018-08-13 15:53:00.874864 127 2033 tokenized/2016/5/16
2018-08-13 15:53:00.875315 128 2033 tokenized/2017/9/25
sentimental score -30
2018-08-13 15:53:01.609809 129 2033 tokenized/2015/1/23
sentimental score 141
2018-08-13 15:53:03.448584 130 2033 tokenized/2016/9/18
sentimental score 27
2018-08-13 15:53:04.440990 131 2033 tokenized/2014/1/27
sentimental score -142
2018-08-13 15:53:06.672171 132 2033 tokenized/2013/2/24
sentimental score -17
2018-08-13 15:53:07.920697 133 2033 tokenized/2015/1/29
sentimental score 153
2018-08-13 15:53:09.624836 134 2033 tokenized/2017/11/10
sentimental score 25
2018-08-13 15:53:10.775940 135 2033 tokenized/2014/12/20
sentimental score 51
2018-08-13 15:53:11.663528 136 2033 tokenized/2017/10/6
sentimental score 40
2018-08-13 

sentimental score 77
2018-08-13 15:54:35.487764 249 2033 tokenized/2017/11/4
sentimental score -46
2018-08-13 15:54:36.136118 250 2033 tokenized/2014/4/23
2018-08-13 15:54:36.136494 251 2033 tokenized/2017/7/7
sentimental score 110
2018-08-13 15:54:37.252155 252 2033 tokenized/2014/1/21
sentimental score -7
2018-08-13 15:54:39.226965 253 2033 tokenized/2014/5/23
sentimental score -112
2018-08-13 15:54:40.758096 254 2033 tokenized/2013/5/15
sentimental score 307
2018-08-13 15:54:43.418021 255 2033 tokenized/2014/1/8
2018-08-13 15:54:43.418417 256 2033 tokenized/2013/11/25
sentimental score -139
2018-08-13 15:54:45.413926 257 2033 tokenized/2013/4/22
sentimental score 82
2018-08-13 15:54:47.681159 258 2033 tokenized/2013/11/6
sentimental score 39
2018-08-13 15:54:49.947291 259 2033 tokenized/2014/5/29
sentimental score 95
2018-08-13 15:54:51.584296 260 2033 tokenized/2017/5/26
sentimental score 138
2018-08-13 15:54:52.616121 261 2033 tokenized/2014/12/13
2018-08-13 15:54:52.616546 262 20

...

sentimental score 95
2018-08-13 16:13:33.125901 1552 2033 tokenized/2016/9/4
2018-08-13 16:13:33.127098 1553 2033 tokenized/2013/3/31
sentimental score -33
2018-08-13 16:13:34.491586 1554 2033 tokenized/2013/12/4
sentimental score 156
2018-08-13 16:13:36.367338 1555 2033 tokenized/2014/1/24
sentimental score 20
2018-08-13 16:13:38.342420 1556 2033 tokenized/2015/9/23
2018-08-13 16:13:38.342869 1557 2033 tokenized/2014/8/22
sentimental score 258
2018-08-13 16:13:40.002762 1558 2033 tokenized/2012/12/31
sentimental score 7
2018-08-13 16:13:40.631733 1559 2033 tokenized/2014/1/20
2018-08-13 16:13:40.632228 1560 2033 tokenized/2016/6/28
sentimental score -33
2018-08-13 16:13:42.234615 1561 2033 tokenized/2017/8/24
sentimental score 103
2018-08-13 16:13:43.470902 1562 2033 tokenized/2016/12/18
2018-08-13 16:13:43.471302 1563 2033 tokenized/2015/3/12
2018-08-13 16:13:43.471655 1564 2033 tokenized/2016/5/15
sentimental score 29
2018-08-13 16:13:44.380908 1565 2033 tokenized/2018/3/16
sentimen

sentimental score -9
2018-08-13 16:15:22.679718 1668 2033 tokenized/2016/2/9
2018-08-13 16:15:22.680344 1669 2033 tokenized/2015/7/21
sentimental score -184
2018-08-13 16:15:24.347039 1670 2033 tokenized/2017/9/28
2018-08-13 16:15:24.347501 1671 2033 tokenized/2013/10/15
sentimental score 293
2018-08-13 16:15:26.876207 1672 2033 tokenized/2016/1/29
sentimental score -97
2018-08-13 16:15:28.526794 1673 2033 tokenized/2013/12/30
2018-08-13 16:15:28.527169 1674 2033 tokenized/2017/10/5
2018-08-13 16:15:28.527625 1675 2033 tokenized/2013/11/28
sentimental score 8
2018-08-13 16:15:30.450349 1676 2033 tokenized/2018/3/31
sentimental score -36
2018-08-13 16:15:31.022381 1677 2033 tokenized/2017/7/3
2018-08-13 16:15:31.022863 1678 2033 tokenized/2018/4/12
2018-08-13 16:15:31.023381 1679 2033 tokenized/2016/12/8
sentimental score 67
2018-08-13 16:15:32.016305 1680 2033 tokenized/2018/4/1
sentimental score -42
2018-08-13 16:15:33.092427 1681 2033 tokenized/2014/7/20
sentimental score 106
2018-08

sentimental score -45
2018-08-13 16:17:07.709784 1786 2033 tokenized/2014/9/1
2018-08-13 16:17:07.710177 1787 2033 tokenized/2018/7/7
2018-08-13 16:17:07.710587 1788 2033 tokenized/2017/11/29
2018-08-13 16:17:07.710972 1789 2033 tokenized/2015/2/23
2018-08-13 16:17:07.712789 1790 2033 tokenized/2016/10/29
2018-08-13 16:17:07.713329 1791 2033 tokenized/2014/10/11
2018-08-13 16:17:07.713731 1792 2033 tokenized/2013/2/26
sentimental score 150
2018-08-13 16:17:09.864097 1793 2033 tokenized/2016/11/2
2018-08-13 16:17:09.864603 1794 2033 tokenized/2017/1/5
sentimental score 130
2018-08-13 16:17:11.025326 1795 2033 tokenized/2016/6/2
sentimental score 54
2018-08-13 16:17:12.249357 1796 2033 tokenized/2014/7/12
sentimental score 110
2018-08-13 16:17:13.046072 1797 2033 tokenized/2018/6/24
2018-08-13 16:17:13.046446 1798 2033 tokenized/2015/2/18
sentimental score 5
2018-08-13 16:17:14.806839 1799 2033 tokenized/2018/1/19
sentimental score 87
2018-08-13 16:17:15.992101 1800 2033 tokenized/2015/1

sentimental score -33
2018-08-13 16:18:38.702334 1903 2033 tokenized/2018/7/3
sentimental score 81
2018-08-13 16:18:39.829603 1904 2033 tokenized/2014/10/20
2018-08-13 16:18:39.830024 1905 2033 tokenized/2018/5/14
sentimental score 179
2018-08-13 16:18:41.043453 1906 2033 tokenized/2014/7/15
sentimental score 205
2018-08-13 16:18:42.561817 1907 2033 tokenized/2013/1/29
2018-08-13 16:18:42.562198 1908 2033 tokenized/2014/10/25
sentimental score 31
2018-08-13 16:18:43.386847 1909 2033 tokenized/2013/7/2
sentimental score -39
2018-08-13 16:18:45.384681 1910 2033 tokenized/2016/10/23
sentimental score -45
2018-08-13 16:18:46.064955 1911 2033 tokenized/2016/3/4
sentimental score 61
2018-08-13 16:18:47.441582 1912 2033 tokenized/2013/11/1
sentimental score -42
2018-08-13 16:18:49.875636 1913 2033 tokenized/2013/12/16
2018-08-13 16:18:49.876737 1914 2033 tokenized/2018/4/24
sentimental score 121
2018-08-13 16:18:51.099015 1915 2033 tokenized/2017/10/21
sentimental score -26
2018-08-13 16:18:5

sentimental score 2
2018-08-13 16:20:31.221072 2017 2033 tokenized/2017/11/24
2018-08-13 16:20:31.223268 2018 2033 tokenized/2016/5/4
sentimental score -21
2018-08-13 16:20:32.401102 2019 2033 tokenized/2015/4/9
2018-08-13 16:20:32.404298 2020 2033 tokenized/2016/8/6
2018-08-13 16:20:32.404806 2021 2033 tokenized/2014/3/4
sentimental score 161
2018-08-13 16:20:33.985671 2022 2033 tokenized/2016/7/30
sentimental score 11
2018-08-13 16:20:34.825943 2023 2033 tokenized/2014/12/8
sentimental score 0
2018-08-13 16:20:36.508883 2024 2033 tokenized/2017/3/5
2018-08-13 16:20:36.509794 2025 2033 tokenized/2016/12/3
2018-08-13 16:20:36.510337 2026 2033 tokenized/2017/5/1
2018-08-13 16:20:36.510718 2027 2033 tokenized/2015/9/21
sentimental score -74
2018-08-13 16:20:37.861321 2028 2033 tokenized/2016/12/4
sentimental score 85
2018-08-13 16:20:38.470419 2029 2033 tokenized/2015/10/20
2018-08-13 16:20:38.470802 2030 2033 tokenized/2018/2/25
2018-08-13 16:20:38.471211 2031 2033 tokenized/2015/1/4
20

In [188]:
import re
re.findall(r'\d{4}\/\d{1,2}\/\d{1,2}', 'tokenized/2018/22/3')

['2018/22/3']

In [256]:
%%time
## create a csv
import glob
ds = glob.glob('tokenized/*/*/*')
dparts = [d.split('/') for d in ds]
dates = [(int(dpart[1]), int(dpart[2]), int(dpart[3])) for dpart in dparts]
dates.sort()


import csv
with open('sentiment.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['date', 'sentiment'])
    writer.writeheader()
    for d in dates:
        path = 'tokenized/{}/{}/{}/trigram.json'.format(d[0], d[1], d[2])
        with open(path) as fr:
            score = int(fr.read())
        writer.writerow({'date' : '{}-{}-{}'.format(d[2], d[1], d[0]), 'sentiment' : score})

CPU times: user 40 ms, sys: 24 ms, total: 64 ms
Wall time: 66.2 ms


# SVR

In [404]:
goodwords = ['ทะยาน', 'วิ่ง','แตะ','แข็ง','เขียว','ขา_ขึ้น','สูง_สุด', 'กำไร', 'แข็งแกร่ง', 'ปั่น', 'ขาย_หมู', 'call', 'long', 'ขาด_ทุน', 'เปิด_l', 'l', 'ลาก']
badwords = ['ดอย', 'หลุด','ร่วง','ตาย','แดง',  'ขา_ลง', 'ต่ำ_สุด', 'คัท', 'หนัก', 'ลบ', 'ตก', 'ร่วง_ลง', 'เลิก', 'เครียด', 'รอด', 'ทุบ', 'กังวล', 'put', 'short', 's', 'อ่อน', 'ไส้_แตก', 'เลือดสาด', 'ตบ', 'ปิดจอ']

In [403]:
wv['ปิด_จอ']

array([-0.17933123,  0.17893074, -0.15398802,  0.46605676, -0.94262594,
        0.516209  , -0.20863159, -0.27593794, -0.0063017 ,  0.09871096,
       -0.515251  ,  0.11251242,  0.48029622,  0.2006212 , -0.09685012,
       -0.4007059 , -0.46740475,  0.37575847, -0.04712586,  0.32536408,
       -0.3418107 ,  0.38607928,  0.2588204 ,  0.32190233,  0.19018143,
        0.5888957 ,  0.56994855,  0.8572202 , -0.35887432,  0.00228762,
        0.905201  , -0.0871786 ,  0.2032396 , -0.33234677,  0.05526485,
       -0.14519966, -0.5481803 ,  0.02682955, -0.04847681,  0.5166316 ,
        0.22186606, -0.60882   ,  0.99551165,  0.5313718 ,  0.35484073,
       -0.08905195, -0.27311736, -0.05452759,  0.24729437,  0.20385322,
       -0.09257133,  0.56201583, -0.21015047,  0.5776392 ,  0.27558258,
       -0.05963071,  0.18523173,  0.20140018, -0.0115049 , -0.34479457,
       -0.42153275, -0.01703904, -0.21173456, -0.3314934 ,  0.127348  ,
       -0.2568982 , -0.03460529, -0.59299254,  0.05060816, -0.09

In [405]:
allwords = goodwords + badwords
trainX = [
    wv[word]
    for word
    in allwords
]
trainY = [1 for word in goodwords] + [0 for word in badwords]


from sklearn import svm
model = svm.SVR()
model.fit(trainX, trainY)

Y = model.predict(trainX)
    
indice = list(range(len(allwords)))
indice.sort(key=lambda i: Y[i])

for i in indice:
    word = allwords[i]
    print('{:5.2f} {}'.format(Y[i], word))

 0.03 หนัก
 0.10 ไส้_แตก
 0.10 ตก
 0.10 กังวล
 0.10 ทุบ
 0.10 ตาย
 0.10 หลุด
 0.10 ดอย
 0.10 ปิดจอ
 0.10 คัท
 0.10 ร่วง_ลง
 0.10 แดง
 0.10 ร่วง
 0.10 เครียด
 0.10 เลือดสาด
 0.11 เลิก
 0.11 ตบ
 0.13 ลบ
 0.13 รอด
 0.18 ต่ำ_สุด
 0.18 s
 0.19 put
 0.20 อ่อน
 0.20 เขียว
 0.22 เปิด_l
 0.22 ขา_ลง
 0.24 ขาย_หมู
 0.24 short
 0.25 l
 0.25 ลาก
 0.25 long
 0.30 ขาด_ทุน
 0.31 call
 0.31 ปั่น
 0.31 แข็ง
 0.37 แตะ
 0.37 วิ่ง
 0.40 สูง_สุด
 0.41 ขา_ขึ้น
 0.46 แข็งแกร่ง
 0.48 กำไร
 0.51 ทะยาน


In [483]:

def svr_sentiment(word):
    X = [wv[word]]
    Y = model.predict(X)
    return Y[0] / 0.5 * 2 - 1

svr_sentiment('ไล่ราคา')

KeyError: "word 'ไล่ราคา' not in vocabulary"

In [484]:
svr_words = list(allwords)
svr_words.sort(key=lambda word: svr_sentiment(word))
for word in svr_words:
    print('{:5.2f} {}'.format(svr_sentiment(word), word))

-0.87 หนัก
-0.60 ไส้_แตก
-0.60 ตก
-0.60 กังวล
-0.60 ทุบ
-0.60 ตาย
-0.60 หลุด
-0.60 ดอย
-0.60 ปิดจอ
-0.60 คัท
-0.60 ร่วง_ลง
-0.60 แดง
-0.60 ร่วง
-0.60 เครียด
-0.60 เลือดสาด
-0.55 เลิก
-0.54 ตบ
-0.50 ลบ
-0.46 รอด
-0.30 ต่ำ_สุด
-0.28 s
-0.22 put
-0.21 อ่อน
-0.18 เขียว
-0.14 เปิด_l
-0.11 ขา_ลง
-0.05 ขาย_หมู
-0.03 short
-0.01 l
 0.00 ลาก
 0.00 long
 0.19 ขาด_ทุน
 0.22 call
 0.23 ปั่น
 0.25 แข็ง
 0.49 แตะ
 0.50 วิ่ง
 0.60 สูง_สุด
 0.63 ขา_ขึ้น
 0.83 แข็งแกร่ง
 0.94 กำไร
 1.06 ทะยาน
